<a href="https://colab.research.google.com/github/dyutivartak/ML-Projects/blob/main/Graduate_Admission_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TASK 1 : Install Dependencies & Run a SparkSession


In [ ]:
#install pyspark
! pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 48 kB/s 
     |████████████████████████████████| 199 kB 60.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=bc1114f31e2005ce330afb18c9d12492ff9270a397676dbc33a1a44682d7dc63
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
#create a sparksession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("spark").getOrCreate()

# TASK 2 : Clone & Explore dataset

In [ ]:
#clone the dataset
! git clone https://github.com/education454/admission_dataset

Cloning into 'admission_dataset'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.


In [ ]:
#check the presence of dataset
! ls admission_dataset

Admission_Predict_Ver1.1.csv


In [ ]:
#create a spark dataframe
df = spark.read.csv('/content/admission_dataset/Admission_Predict_Ver1.1.csv',header=True,inferSchema=True)

In [ ]:
#display dataframe
df.show()

+---------+---------+-----------+-----------------+---+---+----+--------+---------------+
|Serial No|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+---------+-----------+-----------------+---+---+----+--------+---------------+
|        1|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|
|        2|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|
|        3|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72|
|        4|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|
|        5|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|
|        6|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|
|        7|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75|
|        8|      308|        101|                2|3.0|4.0| 7.9|       0|           0.68|
|        9

In [ ]:
#get the no.of rows & columns
print((df.count(),len(df.columns)))

(500, 9)


In [ ]:
#print schema
df.printSchema()

root
 |-- Serial No: integer (nullable = true)
 |-- GRE Score: integer (nullable = true)
 |-- TOEFL Score: integer (nullable = true)
 |-- University Rating: integer (nullable = true)
 |-- SOP: double (nullable = true)
 |-- LOR: double (nullable = true)
 |-- CGPA: double (nullable = true)
 |-- Research: integer (nullable = true)
 |-- Chance of Admit: double (nullable = true)



In [ ]:
#get the summary statistics
df.describe().show()

+-------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|summary|        Serial No|         GRE Score|      TOEFL Score|University Rating|               SOP|               LOR|              CGPA|          Research|    Chance of Admit|
+-------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|  count|              500|               500|              500|              500|               500|               500|               500|               500|                500|
|   mean|            250.5|           316.472|          107.192|            3.114|             3.374|             3.484| 8.576440000000003|              0.56| 0.7217399999999996|
| stddev|144.4818327679989|11.295148372354712|6.081867659564538|1.143511800759815|0.9910036207566072|0.92

# TASK 3 : Data Cleaning

In [ ]:
#drop the unnecessary column
df = df.drop('Serial No')

In [ ]:
#display the dataframe
df.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|
|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|
|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72|
|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|
|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|
|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|
|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75|
|      308|        101|                2|3.0|4.0| 7.9|       0|           0.68|
|      302|        102|                1|2.0|1.5| 8.0|       0|            0.5|
|      323|        108|                3

In [ ]:
#check for null values
for i in df.columns:
  print(i+":",df[df[i].isNull()].count())

GRE Score: 0
TOEFL Score: 0
University Rating: 0
SOP: 0
LOR: 0
CGPA: 0
Research: 0
Chance of Admit: 0


# TASK 4 : Correlation Analysis & Feature Selection

In [ ]:
# correlation analysis
for col in df.columns:
  print('Correlation to chance of admit col for {} is {}'.format(col,df.stat.corr('Chance of Admit',col)))

Correlation to chance of admit col for GRE Score is 0.8103506354632598
Correlation to chance of admit col for TOEFL Score is 0.7922276143050823
Correlation to chance of admit col for University Rating is 0.6901323687886892
Correlation to chance of admit col for SOP is 0.6841365241316723
Correlation to chance of admit col for LOR is 0.6453645135280112
Correlation to chance of admit col for CGPA is 0.882412574904574
Correlation to chance of admit col for Research is 0.5458710294711379
Correlation to chance of admit col for Chance of Admit is 1.0


In [ ]:
# feature selection
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols = ['GRE Score','TOEFL Score','CGPA'],outputCol='features')

In [ ]:
#display dataframe
output_data = assembler.transform(df)
output_data.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+------------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|          features|
+---------+-----------+-----------------+---+---+----+--------+---------------+------------------+
|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|[337.0,118.0,9.65]|
|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|[324.0,107.0,8.87]|
|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72| [316.0,104.0,8.0]|
|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|[322.0,110.0,8.67]|
|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|[314.0,103.0,8.21]|
|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|[330.0,115.0,9.34]|
|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75| [321.0,109.0,8.2]|
|      308

# TASK 5 : Build the Linear Regression Model

In [ ]:
#import Linearregression and create final data
from pyspark.ml.regression import LinearRegression
final_data = output_data.select('features','Chance of Admit')

In [ ]:
#print schema of final data
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Chance of Admit: double (nullable = true)



In [ ]:
#split the dataset into training and testing set
train,test = final_data.randomSplit([0.7,0.3])

In [ ]:
#build & train the model
models = LinearRegression(featuresCol = 'features',labelCol = 'Chance of Admit')
model = models.fit(train)

In [ ]:
#get coefficients & intercept
print("Coefficients: ",model.coefficients)
print("Intercepts: ",model.intercept)

Coefficients:  [0.002482976343498992,0.003972906814943486,0.13415930019839634]
Intercepts:  -1.641908953023515


In [ ]:
#get summary of the model
summary = model.summary

In [ ]:
#print the rmse & r2 score
print("RMSE: ",summary.rootMeanSquaredError)
print("r2: ",summary.r2)

RMSE:  0.06323606198515157
r2:  0.7952745733015681


# TASK 6 : Evaluate & Save the Model

In [ ]:
#transform on the test data
predictions = model.transform(test)

In [ ]:
#display the predictions
predictions.show(20)

+------------------+---------------+------------------+
|          features|Chance of Admit|        prediction|
+------------------+---------------+------------------+
| [294.0,93.0,7.36]|           0.46|0.4449788752151298|
| [295.0,96.0,7.34]|           0.47|0.4566973859994914|
| [296.0,99.0,7.28]|           0.47| 0.463049524775917|
| [296.0,99.0,8.03]|           0.61|0.5636689999247142|
| [297.0,96.0,7.43]|           0.34| 0.473737675704345|
|[298.0,101.0,7.69]|           0.53|0.5309666041741445|
|[298.0,101.0,7.86]|           0.54|0.5537736852078721|
|[299.0,100.0,7.89]|           0.59|0.5563085337423794|
|[299.0,100.0,8.02]|           0.63| 0.573749242768171|
| [300.0,95.0,8.22]|           0.62|0.5831995450766316|
|  [300.0,97.0,8.1]|           0.65| 0.575046242682711|
| [300.0,98.0,8.02]|           0.61|0.5682864054817827|
|  [300.0,99.0,6.8]|           0.36| 0.408584966054683|
|[301.0,100.0,8.04]|           0.67|0.5813983814591366|
|[301.0,104.0,7.89]|           0.68|0.5771661136

In [ ]:
#evaluate the model
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(predictionCol='prediction',labelCol='Chance of Admit',metricName='r2')
print("r2 on test data: ",evaluator.evaluate(predictions))

r2 on test data:  0.8201585443327003


In [ ]:
#save the model
model.save("model")

In [ ]:
#load the model
from pyspark.ml.regression import LinearRegressionModel
model = LinearRegressionModel.load("model")